## Distributed Arrays

In [ ]:
using Distributed

In [ ]:
nprocs() == 1 && addprocs(;exeflags="--project")

In [ ]:
@everywhere using DistributedArrays

In [ ]:
workers()

#### Start with regular array

In [ ]:
dat=rand(10000,10000)

#### Process id 1 is assigned to regular array

In [ ]:
map(x->myid(),dat)

#### Distribute ownership to other workers with distributed array

In [ ]:
ddat=distribute(dat)

#### Different workers are assigned to different blocks of data

In [ ]:
map(x->myid(),ddat)

#### Process each chunk locally by the corresponding worker assigned to the chunk

In [ ]:
[ @spawnat p sum(x->x*x,localpart(ddat)) for p in workers() ]

#### Fetch the results from each worker working on their corresponding chunk of data

In [ ]:
map(fetch,[ @spawnat p sum(x->x*x,localpart(ddat)) for p in workers() ])

#### Finally reduce the returned results from each worker by adding them

In [ ]:
@time reduce(+,map(fetch,[ @spawnat p sum(x->x*x,localpart(ddat)) for p in workers() ]))

In [ ]:
@time sum(ddat .* ddat)

In [ ]:
@code_native sum(ddat .* ddat)

In [ ]:
methodswith(DArray)

In [ ]:
@which sum(ddat .* ddat)